## Generate Interpolations from Stored Vectors

Generating interpolation videos from POM2021 performance. 

Based on [j.mp/wanderclip](https://j.mp/wanderclip) by Eyal Gruss [@eyaler](https://twitter.com/eyaler) [eyalgruss.com](https://eyalgruss.com)

Modified to run on HCC OOD/nautilus.optiputer.net/z8 by robert.twomey@gmail.com

# 0. Run Once

Onetime setup (jupyter environment). Installs necessary python packages. (you may need to restart the kernel after this)

In [15]:
# !pip install perlin-noise
!pip install ipython-autotime
!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 -f https://download.pytorch.org/whl/torch_stable.html ftfy regex
!pip install pytorch-pretrained-biggan
!pip install nltk
!pip install cma

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://download.pytorch.org/whl/torch_stable.html
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
time: 5.78 s (started: 2021-12-23 09:58:28 -06:00)


# 1. Setup and Imports

Setup the timer in case we want it

In [16]:
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 748 µs (started: 2021-12-23 09:58:52 -06:00)


Import necessary libraries

In [17]:
from IPython.display import HTML, clear_output
from PIL import Image
from IPython.display import Image as JupImage
import numpy as np
import nltk
from scipy.stats import truncnorm

# from biggan
import torch
from pytorch_pretrained_biggan import (BigGAN, one_hot_from_names, truncated_noise_sample,
                                       save_as_images, convert_to_images) #, display_in_terminal)
import logging
logging.basicConfig(level=logging.WARNING)

# do we need wordnet?
nltk.download('wordnet')

# load biggan
model = BigGAN.from_pretrained('biggan-deep-512')
print("loaded bigGAN")

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/twomeylab/rtwomey/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


loaded bigGAN
time: 3.52 s (started: 2021-12-23 09:58:55 -06:00)


More imports for interpolation

In [18]:
from numpy import asarray
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from numpy import arccos
from numpy import clip
from numpy import dot
from numpy import sin
from numpy import linspace
from numpy.linalg import norm
import os
import glob

time: 524 µs (started: 2021-12-23 09:58:58 -06:00)


Helper functions for interpolation

In [19]:
# from
# https://discuss.pytorch.org/t/help-regarding-slerp-function-for-generative-model-sampling/32475/4

# spherical linear interpolation (slerp)
def slerp(val, low, high):
    omega = arccos(clip(dot(low/norm(low), high/norm(high)), -1, 1))
    so = sin(omega)
    if so == 0:
        # L'Hopital's rule/LERP
        return (1.0-val) * low + val * high
    return sin((1.0-val)*omega) / so * low + sin(val*omega) / so * high
 
# uniform interpolation between two points in latent space
def interpolate_points(p1, p2, n_steps=10):
    # interpolate ratios between the points
    ratios = np.linspace(0, 1, num=n_steps)
    # linear interpolate vectors
    vectors = list()
    for ratio in ratios:
        v = slerp(ratio, p1, p2)
        vectors.append(v)
    return np.asarray(vectors)

time: 709 µs (started: 2021-12-23 09:59:01 -06:00)


File output settings

In [20]:
import os
# file paths
work = os.getcwd() # get the current path

# No need to seed the interpolation for generation from stored vectors
# np.random.RandomState(1)
# np.random.seed(1)
# torch.manual_seed(1)

# one random variable
truncation = 1.0

# the file directories
workbase = work

# results of text to image translation
resultspath = os.path.join(work, "results/")

# intermediate frames working directory
interpbase = os.path.join(work, "interpolation/")

# video directory
videopath = os.path.join(work, "videos/")

# video and srt output files
# filebase = '{0}'
# moviefilename = filebase+'.mp4'
# srtfilename = filebase+'.srt'

# the interpolation
num_steps = 90#300
len_hold = 30

# frame rate for the the movie
fps = 30

time: 849 µs (started: 2021-12-23 09:59:02 -06:00)


In [21]:
!mkdir -p $interpbase
!mkdir -p $videopath

time: 289 ms (started: 2021-12-23 09:59:02 -06:00)


## 2. Generate Interpolation Between Two Images

In [22]:
def frames_to_TC (frames):
    h = int(frames / 86400) 
    m = int(frames / (60*fps)) % 60 
    s = int((frames % (60*fps))/fps) 
    f = float(frames % (60*fps) % fps)/float(fps)
    return ( "%02d:%02d:%02d,%04d" % ( h, m, s, f))

time: 598 µs (started: 2021-12-23 09:59:06 -06:00)


In [23]:
def make_safe_filename(s):
    def safe_char(c):
        if c.isalnum():
            return c
        else:
            return "_"
    return "".join(safe_char(c) for c in s).rstrip("_")

time: 421 µs (started: 2021-12-23 09:59:10 -06:00)


In [24]:
def generate_video_transition(class_filenames, prompts, interpbase):
    """includes the result with the class_files"""
    
    print("==== retrieve vectors ====")    
    safe_prompt = make_safe_filename(prompts[0])
    noise_filenames = [fname.replace("class", "noise") for fname in class_filenames]

    #print(class_filenames)
    #print(noise_filenames)

    class_inputs = [np.loadtxt(filename) for filename in class_filenames]
    noise_inputs = [np.loadtxt(filename) for filename in noise_filenames]

    # file outputs names and paths
    moviefilename = safe_prompt+'_{0}.mp4'.format(fps)
    srtfilename = safe_prompt+'_{0}.srt'.format(fps)
    interppath = os.path.join(interpbase, safe_prompt)

    !mkdir -p $interppath
    
    print("==== generate interpolations ====")

    count = 0

    for i in range(len(class_inputs)):

        # generate interpolations
        noises = interpolate_points(noise_inputs[i], noise_inputs[(i+1)%len(class_inputs)], num_steps)
        classes = interpolate_points(class_inputs[i], class_inputs[(i+1)%len(class_inputs)], num_steps)

        # generate images in batches
#         batch_size = 10
        batch_size = 40 #RTX8000
    
        for j in range(0, num_steps, batch_size):
            clear_output()
            print(i, j, count)
            noise_vector = noises[j:j+batch_size]
            class_vector = classes[j:j+batch_size]

            # convert to tensors
            noise_vector = torch.tensor(noise_vector, dtype=torch.float32)
            class_vector = torch.tensor(class_vector, dtype=torch.float32)

            # put everything on cuda (GPU)
            noise_vector = noise_vector.to('cuda')
            noise_vector = noise_vector.clamp(-2*truncation, 2*truncation)
            class_vector = class_vector.to('cuda')
            class_vector = class_vector.softmax(dim=-1)
            model.to('cuda')

            # generate images
            with torch.no_grad():
                output = model(noise_vector, class_vector, truncation)

            # If you have a GPU put back on CPU
            output = output.to('cpu')

            imgs = convert_to_images(output)

            # repeat first image for len_hold
            if j == 0:
                for k in range(len_hold):
                    imgs[0].save(interppath+"/output_%05d.png" % count)
                    count = count + 1

            for img in imgs: 
                img.save(interppath+"/output_%05d.png" % count)
                count = count + 1
    
    
    print("==== generating movie ====")
    
    # generate mp4
    outpath = os.path.join(videopath, moviefilename)
    with open('list.txt','w') as f:
      for i in range(count):
        f.write('file %s/output_%05d.png\n'%(interppath, i))
    cmd = "ffmpeg -r {0} -i {1}/output_%05d.png -c:v libx265 -pix_fmt yuv420p -crf 0 -r {0} {2} -y"
    
    print("video creation cmd:\n", cmd.format(fps, interppath, outpath))
#     os.system(cmd.format(fps, interppath, outpath))
    
    print("==== generating subtitles ==== ")
    srtoutpath = os.path.join(videopath, srtfilename)
    with open(srtoutpath,'w') as f:

        frames = 0
        for i in range(len(class_inputs)):
            f.write(str(i+1)+"\n")
            f.write(frames_to_TC(frames)+" --> ")
            frames+=len_hold
            f.write(frames_to_TC(frames)+"\n")
            f.write(prompts[i]+"\n\n")
            frames+=num_steps
    print(frames)

time: 2.08 ms (started: 2021-12-23 09:59:10 -06:00)


In [25]:
prompts = [
    "a drawing of an elegant machine",
    "a machine that learns to make images"
]

latentfiles = [
    "a_drawing_of_an_elegant_machine_class.txt",
    "a_machine_that_learns_to_make_images_class.txt"
]

class_filenames = [resultspath+latent for latent in latentfiles]
    
generate_video_transition(class_filenames, prompts, interpbase)

0 0 0


RuntimeError: CUDA out of memory. Tried to allocate 5.00 GiB (GPU 0; 15.78 GiB total capacity; 11.69 GiB already allocated; 2.42 GiB free; 12.18 GiB reserved in total by PyTorch)

time: 4.87 s (started: 2021-12-23 09:59:11 -06:00)


## 3. Generate Video Interpolating Between Multiple Version of one Image

Helper for generating time codes for subtitles:

In [72]:
def frames_to_TC (frames):
    h = int(frames / 86400) 
    m = int(frames / (60*fps)) % 60 
    s = int((frames % (60*fps))/fps) 
    f = float(frames % (60*fps) % fps)/float(fps)
    return ( "%02d:%02d:%02d,%04d" % ( h, m, s, f))

time: 511 µs (started: 2021-12-21 15:29:11 -06:00)


In [73]:
def make_safe_filename(s):
    def safe_char(c):
        if c.isalnum():
            return c
        else:
            return "_"
    return "".join(safe_char(c) for c in s).rstrip("_")

time: 341 µs (started: 2021-12-21 15:29:12 -06:00)


In [ ]:
def generate_video_hold(output_folder, class_filenames, result_filename, prompt, interpbase, filebase):
    """includes the result with the class_files"""
    
    print("==== retrieve vectors ====")
    
    safe_prompt = make_safe_filename(prompt)

    prompts = [ prompt*(len(class_filenames)+1)]

    class_filenames = [output_folder+file for file in class_filenames]
    class_filenames += [result_filename.format(safe_prompt)]

    noise_filenames = [fname.replace("class", "noise") for fname in class_filenames]

    print(class_filenames)
    print(noise_filenames)

    class_inputs = [np.loadtxt(filename) for filename in class_filenames]
    noise_inputs = [np.loadtxt(filename) for filename in noise_filenames]


    interpbase = interpbase.format(safe_prompt)
    filebase = filebase.format(safe_prompt)
    moviefilename = filebase+'.mp4'
    srtfilename = filebase+'.srt'

    !mkdir -p $interpbase
    
    print("==== generate interpolations ====")

    count = 0
    
    num_inputs = len(class_inputs)
    for i in range(len(class_inputs)):

        # generate interpolations
        noises = interpolate_points(noise_inputs[i], noise_inputs[(i+1)%len(class_inputs)], num_steps)
        classes = interpolate_points(class_inputs[i], class_inputs[(i+1)%len(class_inputs)], num_steps)

        # generate images in batches
#         batch_size = 10 #2080
        batch_size = 60 #RTX8000
        for j in range(0, num_steps, batch_size):
            clear_output()
            print("{0}/{1} clases\t{2}/{3} steps\t{4} frames".format(i, num_inputs, j, num_steps, count))
            noise_vector = noises[j:j+batch_size]
            class_vector = classes[j:j+batch_size]

            # convert to tensors
            noise_vector = torch.tensor(noise_vector, dtype=torch.float32)
            class_vector = torch.tensor(class_vector, dtype=torch.float32)

            # put everything on cuda (GPU)
            noise_vector = noise_vector.to('cuda')
            noise_vector = noise_vector.clamp(-2*truncation, 2*truncation)
            class_vector = class_vector.to('cuda')
            class_vector = class_vector.softmax(dim=-1)
            model.to('cuda')

            # generate images
            with torch.no_grad():
                output = model(noise_vector, class_vector, truncation)

            # If you have a GPU put back on CPU
            output = output.to('cpu')

            imgs = convert_to_images(output)

            # repeat first image

            if j == 0:
                for k in range(len_hold):
                    imgs[0].save(interpbase+"/output_%05d.png" % count)
                    count = count + 1

            for img in imgs: 
                img.save(interpbase+"/output_%05d.png" % count)
                count = count + 1
    
    
    print("==== generating movie ====")
    
    # generate mp4
    out = moviefilename%fps
    with open('list.txt','w') as f:
      for i in range(count):
        f.write('file %s/output_%05d.png\n'%(interpbase, i))
    cmd = "ffmpeg -r {0} -i {1}/output_%05d.png -c:v libx265 -pix_fmt yuv420p -crf 0 -r {0} {2} -y"

    os.system(cmd.format(fps, interpbase, out))
    print(cmd.format(fps, interpbase, out))
    
    print("==== generating subtitles ==== ")
    
    with open(srtfilename%fps,'w') as f:

        frames = 0
        for i in range(len(class_inputs)):
            f.write(str(i+1)+"\n")
            f.write(frames_to_TC(frames)+" --> ")
            frames+=len_hold
            f.write(frames_to_TC(frames)+"\n")
            f.write(prompt+"\n\n")
            frames+=num_steps
    print("subtitles: "+srtfilename%fps)
    print(frames)

In [ ]:
# resultsbase = "/home/jovyan/work/results/results_pom_seed_128/"

# # video and srt output files
# filebase = 'pom_128_%s'
# # filebase = 'pom_10_%s'
# moviefilename = filebase+'.mp4'
# srtfilename = filebase+'.srt'

# prompts = [
# #     "the sublime experience of an iceberg",
# #     "ice is a vastness of possibilities",
# #     "a vastness of spatial dimensions",
# #     "a space of unlimited possibilities that the network must explore",
# #     "a network",
# #     "a network of vast spatial dimensions",
# #     "eighteenth century painting of humans encountering nature",
# #     "a person encountering nature",
# #     "an elegant machine that learns to generate artificial images",
# #     "a drawing of an elegant machine",
# #     "a machine that learns to make images",
#     "a drawing of a machine that learns to make images",
# ]

# for prompt in prompts:
    
#     # prompt = "a silicon mold of a friends hands"

#     safe_prompt = make_safe_filename(prompt)

#     class_filenames = [ 
#         "class_00009.txt", 
#     #     "class_00008.txt", 
#         "class_00007.txt"
#     ]

#     result_filename = resultsbase+"{0}_class.txt"

#     # stored output from synthesis
#     # safe_filename = make_safe_filename(prompt)

#     stored_output_folder = "/home/jovyan/work/process/stored_outputs_pom_seed_128/"+safe_prompt+"/"

#     # stored_output_folder = "/home/jovyan/work/process/stored_outputs_pom_seed_128/"+\
#     #     "a_mushrooms_above_ground_growth_is_the_result_of_a_set_of_instructions_20210914_214118/"

#     interpbase = '/home/jovyan/work/interpolation/pom_interp_128_hold_{0}'
#     filebase = '/home/jovyan/work/interpolation/pom_128_%s_hold_{0}'

#     generate_video_hold(stored_output_folder, class_filenames, result_filename, prompt, interpbase, filebase)
#     print('scp "user@z8.local:/home/user/work/interpolation/pom_128_30_hold_{0}*" .'.format(safe_prompt))

Workaround to download files

In [ ]:
# prompts = [
#     "the sublime experience of an iceberg",
#     "ice is a vastness of possibilities",
#     "a vastness of spatial dimensions",
#     "a space of unlimited possibilities that the network must explore",
#     "a network",
#     "a network of vast spatial dimensions"
#     "eighteenth century painting of humans encountering nature",
#     "a person encountering nature",
#     "an elegant machine that learns to generate artificial images",
#     "a drawing of an elegant machine",
#     "a machine that learns to make images",
#     "a drawing of a machine that learns to make images",
# ]
# for prompt in prompts:
    
#     # prompt = "a silicon mold of a friends hands"

#     safe_prompt = make_safe_filename(prompt)
#     print('scp "user@z8.local:/home/user/work/interpolation/pom_128_30_hold_{0}*" .'.format(safe_prompt))

# Notes


Based on SIREN+CLIP Colabs by: [@advadnoun](https://twitter.com/advadnoun), [@norod78](https://twitter.com/norod78)

Other CLIP notebooks: [OpenAI tutorial](https://colab.research.google.com/github/openai/clip/blob/master/Interacting_with_CLIP.ipynb), [SIREN by @advadnoun](https://colab.research.google.com/drive/1FoHdqoqKntliaQKnMoNs3yn5EALqWtvP), [SIREN by @norod78](https://colab.research.google.com/drive/1K1vfpTEvAmxW2rnhAaALRVyis8EiLOnD), [BigGAN by @advadnoun](https://colab.research.google.com/drive/1NCceX2mbiKOSlAd_o7IU7nA9UskKN5WR), [BigGAN by @eyaler](j.mp/bigclip), [BigGAN by @tg_bomze](https://colab.research.google.com/github/tg-bomze/collection-of-notebooks/blob/master/Text2Image_v2.ipynb), [BigGAN using big-sleep library by @lucidrains](https://colab.research.google.com/drive/1MEWKbm-driRNF8PrU7ogS5o3se-ePyPb), [BigGAN story hallucinator by @bonkerfield](https://colab.research.google.com/drive/1jF8pyZ7uaNYbk9ZiVdxTOajkp8kbmkLK), [StyleGAN2-ADA Anime by @nagolinc](https://colab.research.google.com/github/nagolinc/notebooks/blob/main/TADNE_and_CLIP.ipynb) [v2](https://colab.research.google.com/github/nagolinc/notebooks/blob/main/CLIP_%2B_TADNE_(pytorch)_v2.ipynb)

Using the works:

https://github.com/openai/CLIP

https://tfhub.dev/deepmind/biggan-deep-512

https://github.com/huggingface/pytorch-pretrained-BigGAN

http://www.aiartonline.com/design-2019/eyal-gruss (WanderGAN)

For a curated list of more online generative tools see: [j.mp/generativetools](https://j.mp/generativetools)


# POM Prompts/Themes

We need to generate interpolation videos between each of the prompts in the following groups of themes/prompts. 

### Intro section

hold videos ([onedrive](https://uofnelincoln-my.sharepoint.com/:u:/r/personal/rtwomey2_unl_edu/Documents/POM_AI/pom_videos_hold.zip?csf=1&web=1&e=PF8MQN)):
```
pom_128_30_hold_the_sublime_experience_of_an_iceberg.mp4
pom_128_30_hold_ice_is_a_vastness_of_possibilities.mp4
pom_128_30_hold_a_vastness_of_spatial_dimensions.mp4
pom_128_30_hold_a_space_of_unlimited_possibilities_that_the_network_must_explore.mp4
pom_128_30_hold_a_network.mp4
pom_128_30_hold_a_network_of_vast_spatial_dimensions.mp4
pom_128_30_hold_a_person_encountering_nature.mp4
pom_128_30_hold_eighteenth_century_painting_of_humans_encountering_nature.mp4
pom_128_30_hold_an_elegant_machine_that_learns_to_generate_artificial_images.mp4
pom_128_30_hold_a_drawing_of_an_elegant_machine.mp4
pom_128_30_hold_a_machine_that_learns_to_make_images.mp4
pom_128_30_hold_a_drawing_of_a_machine_that_learns_to_make_images.mp4
```

interpolation videos to be generated:
- one video transitioning between each of the phrases in this list (1->2, 2->3, 3->4, n->1)
- videos of transitions between all pairwise combinations of the list

latent vectors for each phrase are in `results_pom_seed_128.tar.gz` ([onedrive](https://uofnelincoln-my.sharepoint.com/:u:/g/personal/rtwomey2_unl_edu/EVB51YvooN5Fu_0l7jQJhbkB_lx97kcyQ5oBL4Md6Dvvug))

In [3]:
# prompts = [
#     # intro
#     "the sublime experience of an iceberg",
#     "ice is a vastness of possibilities",
#     "the vastness of possibilities of water is not a spatial vastness",
#     "a vastness of spatial dimensions",
#     "a space of unlimited possibilities that the network must explore"
#     "a network",
#     "a network of vast spatial dimensions",
#     "a person encountering nature",
#     "eighteenth century painting of humans encountering nature",
#     "an elegant machine that learns to generate artificial images",
#     "a drawing of an elegant machine",
#     "a machine that learns to make images",
#     "a drawing of a machine that learns to make images",
# ]

### Theme 1 - On Plants, as Potential

hold videos ([onedrive](https://uofnelincoln-my.sharepoint.com/:u:/r/personal/rtwomey2_unl_edu/Documents/POM_AI/pom_videos_hold.zip?csf=1&web=1&e=PF8MQN)):
```
pom_128_30_hold_a_seed__a_plant_s_basic_unit__remains_ever_in_potentia.mp4
pom_128_30_hold_they_make_us_have_arms_and_not_wings.mp4
pom_128_30_hold_a_mushroom_is_the_aerial_manifestation_of_a_larger_infestation.mp4
pom_128_30_hold_growing_mushrooms_in_the_fire_stricken_forests_of_LA.mp4
pom_128_30_hold_growing_mushrooms_in_the_forests_is_a_way_to_uncover_their_potential.mp4
pom_128_30_hold_a_mushrooms_above_ground_growth_is_the_result_of_a_set_of_instructions.mp4
```

interpolation videos to be generated:
- one video transitioning between each of the phrases in this list (1->2, 2->3, 3->4, n->1)
- videos of transitions between all pairwise combinations of the list

latent vectors for each phrase are in `results_pom_seed_128.tar.gz` ([onedrive](https://uofnelincoln-my.sharepoint.com/:u:/g/personal/rtwomey2_unl_edu/EVB51YvooN5Fu_0l7jQJhbkB_lx97kcyQ5oBL4Md6Dvvug))

In [4]:
# prompts = [
#     # theme 1
#     "a mushroom is the aerial manifestation of a larger infestation",
#     "a mushrooms above ground growth is the result of a set of instructions",
#     "growing mushrooms in the fire-stricken forests of LA",
#     "growing mushrooms in the forests is a way to uncover their potential"    
#     "a seed, a plant's basic unit, remains ever in potentia",
#     "they make us have arms and not wings", 
#     "the fire-stricken forests of LA",
#     "the fruiting body of a larger organism"
#]

### Theme 2 - On Our Relationships with Machines

hold videos ([onedrive](https://uofnelincoln-my.sharepoint.com/:u:/r/personal/rtwomey2_unl_edu/Documents/POM_AI/pom_videos_hold.zip?csf=1&web=1&e=PF8MQN)):
```
pom_128_30_hold_a_family_of_robotic_seals.mp4
pom_128_30_hold_nursing_homes_in_japan.mp4
pom_128_30_hold_to_help_the_elderly_cope.mp4
pom_128_30_hold_the_after_effects_of_the_Tsunami.mp4
pom_128_30_hold_the_children_laugh_and_smile.mp4
pom_128_30_hold_their_homes_washed_away.mp4
pom_128_30_hold_watching_the_news_on_television.mp4
pom_128_30_hold_the_glow_of_a_television_screen.mp4
```

interpolation videos to be generated:
- one video transitioning between each of the phrases in this list (1->2, 2->3, 3->4, n->1)
- videos of transitions between all pairwise combinations of the list

latent vectors for each phrase are in `results_pom_seed_128.tar.gz` ([onedrive](https://uofnelincoln-my.sharepoint.com/:u:/g/personal/rtwomey2_unl_edu/EVB51YvooN5Fu_0l7jQJhbkB_lx97kcyQ5oBL4Md6Dvvug))

In [5]:
# prompts = [
#     # theme2
#     "a family of robotic seals",
#     "nursing homes in japan",
#     "to help the elderly cope", 
#     "the after effects of the Tsunami",
#     "the children laugh and smile",
#     "their homes washed away",
#     "watching the news on television",
#     "the glow of a television screen",
#     "spoke through a microphone",
#     "the robotic seals danced"
# ]

### Theme 3 - On Care

hold videos ([onedrive](https://uofnelincoln-my.sharepoint.com/:u:/r/personal/rtwomey2_unl_edu/Documents/POM_AI/pom_videos_hold.zip?csf=1&web=1&e=PF8MQN)):
```
pom_128_30_hold_an_xray_of_the_first_silicon_chip.mp4            
pom_128_30_hold_the_dover_demonstration_chip.mp4
pom_128_30_hold_a_picture_of_his_arthritic_hands.mp4
pom_128_30_hold_a_piece_of_wire_to_connect_two_silicon_squares.mp4
pom_128_30_hold_smaller_and_smaller_computer_chips.mp4
pom_128_30_hold_the_kilby_diode.mp4
pom_128_30_hold_a_silicon_mold_of_a_friends_hands.mp4
pom_128_30_hold_in_these_containers__we_care_for_machines.mp4
```

interpolation videos to be generated:
- one video transitioning between each of the phrases in this list (1->2, 2->3, 3->4, n->1)
- videos of transitions between all pairwise combinations of the list

latent vectors for each phrase are in `results_pom_seed_128.tar.gz` ([onedrive](https://uofnelincoln-my.sharepoint.com/:u:/g/personal/rtwomey2_unl_edu/EVB51YvooN5Fu_0l7jQJhbkB_lx97kcyQ5oBL4Md6Dvvug))

In [6]:
# prompts = [
#     # theme 3
#     "an xray of the first silicon chip",
#     "the dover demonstration chip",
#     "a picture of his arthritic hands",
#     "a piece of wire to connect two silicon squares",
#     "smaller and smaller computer chips",
#     "the kilby diode",
#     "a silicon mold of a friends hands"
#     "in these containers, we care for machines",
# ]

### Theme 4 - On Extraction

hold videos ([onedrive](https://uofnelincoln-my.sharepoint.com/:u:/r/personal/rtwomey2_unl_edu/Documents/POM_AI/pom_videos_hold.zip?csf=1&web=1&e=PF8MQN)):
```
pom_128_30_hold_the_first_computing_devices_were_powered_by_steam_engines.mp4
pom_128_30_hold_fueled_with_the_burning_of_fossilized_coal.mp4
pom_128_30_hold_the_bones_of_sparse_data_structures.mp4
pom_128_30_hold_the_slow_demise_of_our_geological_resources.mp4
pom_128_30_hold_extracting_lithium.mp4
pom_128_30_hold_battery_technology.mp4
pom_128_30_hold_computing_devices_can_be_charged_without_being_plugged_in.mp4
pom_128_30_hold_key_contributor_to_climate_change.mp4
pom_128_30_hold_we_are_powered_by_imagination.mp4
pom_128_30_hold_the_machines_we_use_to_think.mp4
pom_128_30_hold_the_machines_that_are_destroying_the_planet.mp4
```

interpolation videos to be generated:
- one video transitioning between each of the phrases in this list (1->2, 2->3, 3->4, n->1)
- videos of transitions between all pairwise combinations of the list

latent vectors for each phrase are in `results_pom_seed_128.tar.gz` ([onedrive](https://uofnelincoln-my.sharepoint.com/:u:/g/personal/rtwomey2_unl_edu/EVB51YvooN5Fu_0l7jQJhbkB_lx97kcyQ5oBL4Md6Dvvug))

In [8]:
# prompts = [
#     #theme 4
#     "the first computing devices were powered by steam engines",
#     "fueled with the burning of fossilized coal",
#     "the bones of sparse data structures",
#     "the slow demise of our geological resources",
#     "extracting lithium",
#     "battery technology",
#     "computing devices can be charged without being plugged in",
#     "key contributor to climate change",
#     "we are powered by imagination",
#     "the machines we use to think",
#     "the machines that are destroying the planet"
# ]

### Theme 5 - On Beauty, Aesthetics, and the Sublime

hold videos ([onedrive](https://uofnelincoln-my.sharepoint.com/:u:/r/personal/rtwomey2_unl_edu/Documents/POM_AI/pom_videos_hold.zip?csf=1&web=1&e=PF8MQN)):
```
pom_128_30_hold_a_painting_of_pygmalion.mp4
pom_128_30_hold_a_sculptor_who_fell_in_love_with_his_own_sculpture.mp4
pom_128_30_hold_everyone_who_saw_her_would_fall_in_love_with_her.mp4 
pom_128_30_hold_beautiful_reflections_of_our_digital_twinse.mp4 
pom_128_30_hold_beautiful_reflections_of_our_digital_twins.mp4 
pom_128_30_hold_a_statue_of_a_woman.mp4
pom_128_30_hold_a_living_breathing_woman.mp4
pom_128_30_hold_our_digital_twins__artificial_intelligence.mp4 
pom_128_30_hold_a_projection_of_our_deepest_fears.mp4
```

interpolation videos to be generated:
- one video transitioning between each of the phrases in this list (1->2, 2->3, 3->4, n->1)
- videos of transitions between all pairwise combinations of the list

latent vectors for each phrase are in `results_pom_seed_128.tar.gz` ([onedrive](https://uofnelincoln-my.sharepoint.com/:u:/g/personal/rtwomey2_unl_edu/EVB51YvooN5Fu_0l7jQJhbkB_lx97kcyQ5oBL4Md6Dvvug))

In [9]:
# prompts = [
#     # theme 5
#     "a painting of pygmalion",
#     "a sculptor who fell in love with his own sculpture",
#     "everyone who saw her would fall in love with her",
#     "beautiful reflections of our digital twinse",
#     "a statue of a woman",
#     "a living breathing woman",
#     "our digital twins, artificial intelligence",
#     "a projection of our deepest fears"
# ]